## Top N Recommender

In [1]:
from collections import defaultdict
from surprise import Dataset
import pandas as pd
from surprise import SVD
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import numpy as np
from surprise import dump
import os
from surprise.model_selection import KFold
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import get_dataset_dir
from matplotlib import pyplot as plt


In [2]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

### Web Crawling

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
from time import sleep
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import time
import random


def pre_process():
    cookie= '''x-wl-uid=1gIJzK3ob9znN4x1aBBMZXMDb5Y5NSAV5EtljKOAqkgJ5ggTheHiJRX51hvKAVaeTNqOjJhKCbA0=; session-id=147-0412936-2984026; ubid-main=131-5601909-3137004; session-id-time=2082787201l; session-token="i/maF0ulOY1dClH++bonSg8Y+QYCTvdyQ6VlWBxFja7ZsWQ1RMf8wiqpjqyrzNA80fJKewRw2BnzGEv0Jv2QY6rNoYZsjudb8d+WuENROCCxJ67rE0lRg+9uxO6wqvC3IlVmwwmZ3A5JbnVfz9NaIHqQPR0qfdsOpEJUGgsByWJySnqHBTZFMXqpdto+NDQIvTGC6AHWaFHOZc5jeJNZnvDIVCTujIobxJn9Xy/MTVVCos6Q5MlQvw=="; i18n-prefs=USD; sp-cdn="L5Z9:NZ"'''
    header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
    'Connection': 'keep-alive',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cookie': cookie}

    df = pd.read_csv('./testForInput.csv')
    item_ids = df['itemID'].unique()

    # items_detail = pd.DataFrame(columns=("id", "item_name", "item_url"))

    items_detail = pd.read_csv('items_detail_without_id.csv')


def get_items_name():
    print(111)
    url = "https://www.amazon.com/dp/"
    # for item_id in item_ids:
    for item_id in item_ids[:2]:
        item_url = url + str(item_id);
        print(item_url)
        html = requests.get(item_url, headers=header).text
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.find("title")
        print(title.text)
        if (title.text != "Sorry! Something went wrong!"):
            item_name = title.text.split(":")[1].strip()
            print(item_name)
            items_detail = items_detail.append(
                pd.DataFrame({'id': [item_id], 'item_name': [item_name], 'item_url': [item_url]}), ignore_index=True)
        else:
            items_detail = items_detail.append(pd.DataFrame({'id': [item_name], 'item_name': ["None"], 'item_url': ["None"]}),
                                               ignore_index=True)
            err = err + 1

        time.sleep(random.randint(3, 7))

    # items_detail.to_csv('items_detail.csv', encoding='utf-8', index=False)
def get_items_detail():
    new_items_detail = pd.DataFrame(columns=("item_id", "item_name", "img_url"))

    for index, item in items_detail.iloc[:2].iterrows():
    # for index, item in items_detail.iloc[:3887].iterrows():
        if (len(item_ids) > index):
            item_id = item_ids[index]
            print(index, item_id, item['item_url'])
            html = requests.get(item['item_url'], headers=header).text
            soup = BeautifulSoup(html, 'html.parser')
            if (soup.find("img", class_="aligncenter") == None):
                img = "None"
            else:
                img = soup.find("img", class_="aligncenter")['src']

            new_items_detail = new_items_detail.append(pd.DataFrame({'item_id': [item_id],
                                                                     'item_name': [item['item_name']], 'img_url': [img]
                                                                     }), ignore_index=True)

    print(new_items_detail.shape)
    print(new_items_detail.head())
    new_items_detail.to_csv('items_detail_test_demo.csv', encoding='utf-8', index=False)


def get_tiems_name2():
    url = "https://igg-games.com/list-215968962-game.html"
    html = requests.get(url, headers=header).text
    soup = BeautifulSoup(html, 'html.parser')
    a_list = soup.find("div",attrs={"property":"text"}).find_all("a", attrs={"rel": "noopener noreferrer"})
    items_detail = pd.DataFrame(columns=("id", "item_name", "item_url"))

    for a in a_list:
        print(a.text)
        item_name = a.text
        item_url = a['href']
        print(a['href'])
        items_detail = items_detail.append(pd.DataFrame({'item_name': [item_name], 'item_url': [item_url]}),
                                           ignore_index=True)
    items_detail.to_csv('items_detail_without_id.csv', encoding='utf-8', index=False)
def get_items_detail2():
    items_detail = pd.read_csv('items_detail_without_id.csv')
    new_items_detail = pd.DataFrame(columns=("item_id", "item_name", "img_url"))

    for index, item in items_detail.iloc[:3887].iterrows():
        if (len(item_ids) > index):
            item_id = item_ids[index]
            print(index, item_id, item['item_url'])
            html = requests.get(item['item_url'], headers=header).text
            soup = BeautifulSoup(html, 'html.parser')
            if (soup.find("img", class_="aligncenter") == None):
                img = "None"
            else:
                img = soup.find("img", class_="aligncenter")['src']

            new_items_detail = new_items_detail.append(pd.DataFrame({'item_id': [item_id],
                                                                     'item_name': [item['item_name']], 'img_url': [img]
                                                                     }), ignore_index=True)

    print(new_items_detail.shape)
    print(new_items_detail.head())
    new_items_detail.to_csv('items_detail_test.csv', encoding='utf-8', index=False)

#if __name__ == '__main__':
    #pre_process()
    #get_tiems_name2()
    #get_items_detail2()